In [ ]:
! pip install -U fastai

In [ ]:
from fastai import *
from fastai.text import *

In [ ]:
! kaggle competitions download -c quora-insincere-questions-classification -f train.csv
! kaggle competitions download -c quora-insincere-questions-classification -f test.csv
! mkdir data/quora
! unzip train.csv.zip -d data/
! unzip test.csv.zip -d data/
! chmod a+r data/*

In [ ]:
path = './data/'

In [ ]:
df = pd.read_csv(path+'train.csv')
df.head()

In [ ]:
sample = df.sample(50000)
sample.to_csv(path+'quora_sample.csv', index=False)
sample.head()

In [ ]:
lm_data = TextLMDataBunch.from_csv(Path(path), 'quora_sample.csv', text_cols='question_text')

In [ ]:
lm_data.show_batch()

In [ ]:
lm_data.save('lmdata')

In [ ]:
learn = language_model_learner(lm_data, drop_mult=0.3, pretrained_model=URLs.WT103)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7), wd=0.03)

In [ ]:
learn.predict(text="Why is the first", n_words=20)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7), wd=0.03, pct_start=0.25)

In [ ]:
learn.save('lm_stage2')

In [ ]:
learn.save_encoder('lm_encoder_stage2')

In [ ]:
learn.load('lm_stage2')

In [ ]:
data_clas = TextClasDataBunch.from_csv(path, 'quora_sample.csv', label_cols='target', text_cols='question_text')

In [ ]:
data_clas.show_batch()

In [ ]:
data_clas.save('clasdata')

In [ ]:
learn = text_classifier_learner(data_clas, drop_mult=0.4)
learn.load_encoder('lm_encoder_stage2')
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))

In [ ]:
learn.save('first')

In [ ]:
learn.load('first')

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/2.6,1e-2), moms=(0.8,0.7), pct_start=0.1)

In [ ]:
learn.save('second')

In [ ]:
learn.load('second')

In [102]:
learn.get_preds()

[tensor([[0.9693, 0.0307],
         [0.9475, 0.0525],
         [0.9468, 0.0532],
         ...,
         [0.9880, 0.0120],
         [0.9324, 0.0676],
         [0.9235, 0.0765]]), tensor([0, 0, 0,  ..., 0, 0, 1])]